## Remove hands on 3D human model

***
`model.obj` and `joints.obj` files should be placed in same folder as this Ipython Notebook.

Functions overview:
 1. `load_model(model_URL)` <br> loads mesh in memory. <br>
 2. `load_joints(joints_URL)` <br> loads joints in memory. <br>
 3. `write_obj(points, faces=[], path='no_hand.obj')` <br> saves result as `obj` file. <br>
 4. `remove_hands(m_points, m_faces, sorted_joints)` <br> removes hand's faces from elbow and smooths borders. <br>
 5. `remove_extra_points(m_points, m_faces)` <br> removes extra points not used in faces.

In [1]:
import numpy as np
import copy
import cProfile

import plotly.offline as py
import plotly.graph_objects as go

py.init_notebook_mode(connected=True)

In [2]:
def load_model(model_URL):
    """ Load model obj file. Returns two 3D np.arrays: points coordinates and faces. """
    #print('Loading model from "{}"...'.format(model_URL))
    m_points = []
    m_faces = []
    with open(model_URL, 'r') as f:
        lines = f.read() 
    for line in lines.split('\n'):
        try:
            v, x, y, z = line.split(' ')
        except ValueError:
            continue
        if v == 'v':
            m_points.append([float(x), float(y), float(z)])
        elif v == 'f':
            m_faces.append([int(x), int(y), int(z)])
    return np.array(m_points), np.array(m_faces)

def load_joints(joints_URL):
    """ Load joints as a np.array of coordinates. """
    #print('Loading joints from "{}"...'.format(joints_URL))
    j_points = []
    with open(joints_URL, 'r') as f:
        lines = f.read() 
    for line in lines.split('\n'):
        try:
            v, x, y, z = line.split(' ')
        except ValueError:
            continue
        j_points.append([float(x), float(y), float(z)])
    return np.array(j_points)

def write_obj(points, faces=[], path='no_hand.obj'):
    """ Save array of points and faces as an obj file to a path. """
    #print('Saving file to path "{}"...'.format(path))
    with open(path, 'w') as f:
        for line in points:
            f.write('v {0[0]} {0[1]} {0[2]}\n'.format(line))
        for line in faces:
            f.write('f {0[0]} {0[1]} {0[2]}\n'.format(line))

def remove_hands(m_points, m_faces, m_joints):
    """ Removes faces of both hands.
    
    Plane vector direction is set as shoulder-wrist joints vector.
    Each hand removing is done using two planes determined by ed_vector and side_vector.
    Smoothing is done by shifting out-of-border point positions (projecting to the elbow plane)
    """
    #print('Removing hands...')
    
    sorted_joints = np.array(sorted(m_joints.tolist()))
    elbow_l, elbow_r = sorted_joints[1], sorted_joints[-2]  # elbow is second joint by Ox axis
    l_ed_vector = ((sorted_joints[0] - sorted_joints[2]) / 
                       np.linalg.norm(sorted_joints[0] - sorted_joints[2]))  # wrist joint and shoulder joint
    r_ed_vector = ((sorted_joints[-1] - sorted_joints[-3]) / 
                       np.linalg.norm(sorted_joints[-1] - sorted_joints[-3]))
    side_vector = np.array([1, 0, 0])
    
    good_faces = []
    l_good_indexes = set()
    r_good_indexes = set()
    l_border_faces = []
    r_border_faces = []
    border_indexes = set()
    
    # choosing good points indexes, that are out of one of two planes or in right direction 
    # (distance_l < 0, side > -0.225)
    for i in range(1, len(m_points)+1):
        point = m_points[i-1]
        distance_l = np.vdot(point - elbow_l, l_ed_vector)
        distance_r = np.vdot(point - elbow_r, r_ed_vector)
        side = np.vdot(point, side_vector)
        if distance_l < 0 or side > -0.225:
            l_good_indexes.add(i)
        if  distance_r < 0 or side < 0.225:
            r_good_indexes.add(i)
    
    # picking good and border faces using good point indexes
    for face in m_faces:
        if (set(face) > l_good_indexes) or (set(face) > r_good_indexes):
            good_faces.append(face)
        elif set(face) & l_good_indexes and (set(face) < r_good_indexes):
            l_border_faces.append(face)
        elif set(face) & r_good_indexes and (set(face) < l_good_indexes):
            r_border_faces.append(face)
        else: 
            pass
    
    #print('Smoothing hands...')
    good_faces += l_border_faces
    good_faces += r_border_faces

    # making set of border points to project
    l_border_indexes = set(np.array(l_border_faces).flatten()) - set(l_good_indexes)
    r_border_indexes = set(np.array(r_border_faces).flatten()) - set(r_good_indexes)
    
    # projecting on the elbows planes
    for l_b_index in l_border_indexes:
        delta = np.vdot(m_points[l_b_index-1] - elbow_l, l_ed_vector) * l_ed_vector
        m_points[l_b_index-1] -= delta  
    
    for r_b_index in r_border_indexes:
        delta = np.vdot(m_points[r_b_index-1] - elbow_r, r_ed_vector) * r_ed_vector
        m_points[r_b_index-1] -= delta

    return m_points, np.array(good_faces)

def remove_extra_points(m_points, m_faces):
    """ Remove points that are not used in faces, reindexing. """
    #print('Removing extra points...')
    good_indexes = set()
    good_points = []
    dic = dict()
    
    #creating a set of points indexes used in faces
    good_indexes = set(m_faces.flatten())
    good_indexes_list = np.array(sorted(list(good_indexes)))
    
    # adding good points to a new list and creatind a reindexing dictionary
    mp = np.arange(0, max(good_indexes_list))
    for i in range(len(good_indexes_list)):
        good_points.append(m_points[good_indexes_list[i]-1])
        dic[good_indexes_list[i]] = i+1
        
    # reindexing faces for good points
    for i in range(len(m_faces)):
        for j in range(3):
            m_faces[i][j] = dic[m_faces[i][j]]
    return good_points, m_faces            

def show3d(mesh, show_verts=True, show_faces=True):
    """ Show 3D object using plotly (interactive visualization). """
    
    print(len(mesh[0]), mesh[1].max())
    
    axis = dict(
        showbackground=True,
        backgroundcolor="rgb(230, 230,1000)",
        gridcolor="rgb(255, 130, 255)",
        zerolinecolor="rgb(120, 255, 255)",
    )
    xaxis = copy.deepcopy(axis)
    xaxis.update(dict(range=[-0.6,0.6]))
    yaxis = copy.deepcopy(axis)
    yaxis.update(dict(range=[-0.5,0.5]))
    zaxis = copy.deepcopy(axis)
    zaxis.update(dict(range=[-1,1]))
    
    layout = go.Layout(
        legend=dict(x=1, y=0.5),
        showlegend=True,
        margin = dict(
        r = 10,
        t = 0,
        b = 0,
        l = 10
      ),
        width=scr_x,
        height=scr_y,
        scene=dict(
        camera=dict(
            eye=dict(x=0.6, y=2, z=0.3)),
            xaxis=xaxis,
            yaxis=yaxis,
            zaxis=zaxis,
            aspectratio=dict(
                x=0.8, y=0.7, z=1.2
            ),
        )
    )

    data = list()
    x, y, z = np.array(mesh[0]).T
    i, j, k = np.array(mesh[1]).T-1
    if show_verts == True:
        data.append(go.Scatter3d(x=x, y=z, z=y, mode='markers', marker=dict(size=0.7)))
    if show_faces == True:
        data.append(go.Mesh3d(x=x, y=z, z=y, i=i, j=k, k=j, opacity=0.5))
    
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)    

In [3]:
# Execution time on Intel 7300HQ (Lenovo IdeaPad 720s-15ikb): 208 ms ± 10.6 ms

model_URL = 'model.obj'
joints_URL = 'joints.obj'
result_URL = 'no_hands.obj'  # where to save model with no hands

scr_x = 800  # Figure width
scr_y = scr_x  # Figure height


def main(show=True):
    
    m_points, m_faces = load_model(model_URL)
    m_joints = load_joints(joints_URL)
    if show:
        show3d([m_points, m_faces])
        
    no_hands = remove_hands(m_points, m_faces, m_joints)
    result = remove_extra_points(*no_hands)
    if show:
        show3d(result)
        
    write_obj(*result, path=result_URL)


main(show=True)

6449 6449


5809 5809


In [6]:
%timeit main(show=False)

199 ms ± 4.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
cProfile.run('main(False)')

         161345 function calls (161343 primitive calls) in 0.255 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(dot)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(norm)
    19399    0.008    0.000    0.031    0.000 <__array_function__ internals>:2(vdot)
        1    0.019    0.019    0.039    0.039 <ipython-input-2-90fbfdd41369>:1(load_model)
        1    0.031    0.031    0.032    0.032 <ipython-input-2-90fbfdd41369>:108(remove_extra_points)
        1    0.000    0.000    0.000    0.000 <ipython-input-2-90fbfdd41369>:19(load_joints)
        1    0.005    0.005    0.033    0.033 <ipython-input-2-90fbfdd41369>:33(write_obj)
        1    0.109    0.109    0.148    0.148 <ipython-input-2-90fbfdd41369>:42(remove_hands)
        1    0.003    0.003    0.255    0.255 <ipython-input-3-8d0db7263cb5>:11(main)
        1    